In [ ]:
import os
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Directory containing waveforms
directory = ''
file_pattern = "100m_no_occlusion_tx_{:02d}.txt"
num_files = 64

# Define segments with their labels
# segments = [ #no occlusion 100m
#     (0, 5, "SF7"), (5, 15, "SF8"), 
#     (19, 65, "SF10"), (65, 235, "SF12"), 
#     (237, 244, "CR2"), (252, 261, "CR3"), 
#     (261, 268, "CR4"), (268, 274, "Tx1"), 
#     (274, 280, "Tx3"), (280, 286, "Tx5")
# ]

# segments = [ #for copper 100m
#     (0, 0.97, "SF: 7"), (0.97, 10.7, "SF: 8"), 
#     (10.7, 60.4, "SF: 10"), (60.4, 233, "SF: 12"), 
#     (233, 241.1, "CR: 2"), (241.1, 249.5, "CR: 3"), 
#     (249.5, 258.3, "CR: 4"), (264.9, 271, "Tx Power: 3"), 
#     (271, 277.4, "Tx Power: 5"), (277.4, 283.8, "Tx Power: 1"),
#     (283.8, 290, 'Tx Power: 14')
# ]

segments = [ #for 400m
    # (0, 0.97, "SF: 7"), (0.97, 10.7, "SF: 8"), 
    # (10.7, 60.4, "SF: 10"), 
    (0, 71.7, "SF: 12"), 
    (71.7, 79.3, "CR: 2"), (79.3, 86.8, "CR: 3"), 
    (86.8, 89.3, "CR: 4")
]


def read_and_downsample(filepath, downsample_factor=5):
    data = pd.read_csv(filepath, sep='\t', header=0, skiprows=3)
    # Ensure we only keep the first two columns
    data = data.iloc[::downsample_factor, :2]
    # Rename columns to standard names
    data.columns = ['Time', 'Voltage']
    return data

def combine_all_data():
    all_data = []
    cumulative_time_shift = 0
    
    for i in range(1, num_files + 1):
        file_name = file_pattern.format(i)
        file_path = os.path.join(directory, file_name)
        
        if os.path.isfile(file_path):
            data = read_and_downsample(file_path)
            
            if not data.empty:
                # Shift time values
                data['Time'] += cumulative_time_shift
                
                all_data.append(data)
                
                time_step = data['Time'].iloc[1] - data['Time'].iloc[0]
                cumulative_time_shift = data['Time'].iloc[-1] + time_step
        else:
            print(f"File {file_name} not found.")
    
    # Combine all data
    combined_data = pd.concat(all_data, ignore_index=True)
    return combined_data

def plot_segments(data, segments):
    for start_time, end_time, label in segments:
        mask = (data['Time'] >= start_time) & (data['Time'] <= end_time)
        segment_data = data[mask].copy()
        
        if not segment_data.empty:
            print(f"Plotting segment {label} ({start_time}-{end_time}s)")
            print(f"Number of points in segment: {len(segment_data)}")
            
            fig = go.Figure()
            
            fig.add_trace(go.Scatter(
                x=segment_data['Time'],
                y=segment_data['Voltage'],
                mode='lines',
                name=label
            ))
            
            fig.update_layout(
                title=f'Segment: {label} (Time: {start_time}-{end_time}s)',
                xaxis_title='Time (s)',
                yaxis_title='Voltage (V)',
                showlegend=True
            )
            
            fig.show()
        else:
            print(f"No data found for segment {label} ({start_time}-{end_time}s)")

def main():
    print("Combining all data...")
    combined_data = combine_all_data()
    
    print(f"Combined data shape: {combined_data.shape}")
    print("Time range:", combined_data['Time'].min(), "to", combined_data['Time'].max())
    
    # Save to CSV
    output_file = "combined_waveform_data.csv"
    combined_data.to_csv(output_file, index=False)
    print(f"Combined data saved to {output_file}")
    
    # Create individual plots for each segment
    print("Creating segment plots...")
    plot_segments(combined_data, segments)

if __name__ == "__main__":
    main()

In [ ]:
import os
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import scipy.signal
from scipy.signal import medfilt

# Directory containing waveforms
directory = ''
file_pattern = "100m_no_occlusion_tx_{:02d}.txt"
num_files = 64
kernel = 21
poles = 7
cutoff = 30
resistor = 18
voltage = 3.3

# Define segments
segments = [ #for no occlusion 100m
    (0, 5, "SF: 7"), (5, 15, "SF: 8"), 
    (19, 65, "SF: 10"), (65, 236.6, "SF: 12"), 
    (237, 244, "CR: 2"), (252.2, 261.3, "CR: 3"), 
    (261.3, 268, "CR: 4"), (268, 274, "Tx Power: 3"), 
    (274, 280.2, "Tx Power: 5"), (280.2, 286, "Tx Power: 1"),
    (286.5, 293, 'Tx Power: 14')
]

# segments = [ #for copper 100m
#     (0, 0.97, "SF: 7"), (0.97, 10.7, "SF: 8"), 
#     (10.7, 60.4, "SF: 10"), (60.4, 233, "SF: 12"), 
#     (233, 241.1, "CR: 2"), (241.1, 249.5, "CR: 3"), 
#     (249.5, 258.3, "CR: 4"), (264.9, 271, "Tx Power: 3"), 
#     (271, 277.4, "Tx Power: 5"), (277.4, 283.8, "Tx Power: 1"),
#     (283.8, 290, 'Tx Power: 14')
# ]

# segments = [ #for 400m
#     # (0, 0.97, "SF: 7"), (0.97, 10.7, "SF: 8"), 
#     # (10.7, 60.4, "SF: 10"), 
#     (0, 71.7, "SF: 12"), 
#     (71.7, 79.3, "CR: 2"), (79.3, 86.8, "CR: 3"), 
#     (86.8, 99.3, "CR: 4")
# ]

def read_and_downsample(filepath, downsample_factor=10):
    data = pd.read_csv(filepath, sep='\t', header=0, skiprows=3)
    data = data.iloc[::downsample_factor, :2]
    # Rename columns
    data.columns = ['Time', 'Voltage']
    data['Voltage'] = data['Voltage']

    #Filter data
    filtered_data = data.copy()
    sample_rate = 1 / np.mean(np.diff(data['Time']))
    sos = scipy.signal.butter(poles, cutoff, 'lowpass', fs=sample_rate, output='sos')
    filtered_data['Voltage'] = scipy.signal.sosfiltfilt(sos, filtered_data['Voltage'])
    filtered_data['Voltage'] = medfilt(data['Voltage'], kernel)

    #Calculate current from voltage given value of shunt resistor
    filtered_data['Current'] = filtered_data['Voltage']/resistor

    return filtered_data

def combine_all_data():
    all_data = []
    cumulative_time_shift = 0
    
    for i in range(1, num_files + 1):
        file_name = file_pattern.format(i)
        file_path = os.path.join(directory, file_name)
        
        if os.path.isfile(file_path):
            data = read_and_downsample(file_path)
            
            if not data.empty:
                data['Time'] = data['Time'] + cumulative_time_shift

                all_data.append(data)
                
                # Update cumulative_time_shift for next file
                cumulative_time_shift = data['Time'].max()
                
                # Add gap between files to prevent overlap
                cumulative_time_shift += 0.0001
                
            else:
                print(f"Empty data in file {file_name}")
        else:
            print(f"File {file_name} not found.")
    
    # Combine all data
    combined_data = pd.concat(all_data, ignore_index=True)
    combined_data = combined_data.sort_values('Time').reset_index(drop=True)
    
    return combined_data

def plot_segments(data, segments):
    # threshold for current changes 
    threshold = 0.01

    # column for detecting when current changes beyond threshold
    data['current_change'] = data['Current'].diff().abs() > threshold

    # Cumulative sum to group sections with the same current level
    data['current_group'] = data['current_change'].cumsum()

    # Calculate duration (in ms) spent at each current level
    grouped = data.groupby('current_group').agg(
        current=('Current', 'mean'),
        start_time=('Time', 'min'),
        end_time=('Time', 'max')
    ).reset_index()

    # Calculate the duration in milliseconds for each current level
    grouped['duration_ms'] = (grouped['end_time'] - grouped['start_time']) * 1000
    # Remove rows where duration is 0
    grouped = grouped[grouped['duration_ms'] > 0]
    grouped.reset_index(drop=True, inplace=True)


    for start_time, end_time, label in segments:
        mask = (data['Time'] >= start_time) & (data['Time'] <= end_time)
        segment_data = data[mask].copy()

        segment_grouped = grouped[(grouped['start_time'] >= start_time) & (grouped['end_time'] <= end_time)].copy()
        segment_grouped['Power'] = segment_grouped['current'] * voltage #current * voltage supply
        segment_grouped['Energy'] = segment_grouped['Power'] * segment_grouped['duration_ms']/1000 #in Watt/seconds or Joules
        
        if not segment_grouped.empty:
            print(segment_grouped[['current', 'duration_ms', 'Power', 'Energy']].to_csv(index=True))
            total_energy = segment_grouped['Energy'].sum()
            total_duration = segment_grouped['duration_ms'].sum()

            print(f"Total duration: {total_duration:.5f} ms\n")
            print(f"Total energy: {total_energy:.5f} J (or Ws)\n")
        else:
            "No current data"

        if not segment_data.empty:  

            try:
                fig = go.Figure()
                
                trace = go.Scatter(
                    x=segment_data['Time'].values,
                    y=segment_data['Current'].values,
                    mode='lines',
                    name=label
                )
                
                fig.add_trace(trace)
                
                fig.update_layout(
                    title=f'Test: {label}',
                    xaxis_title='Time (s)',
                    yaxis_title='Current (A)',
                    showlegend=True
                )
                
                fig.show()
                
            except Exception as e:
                print(f"Error creating/showing plot for segment {label}: {str(e)}")
                print("Data types:")
                print(segment_data.dtypes)
                print("\nAny NaN values?")
                print(segment_data.isna().sum())
                
        else:
            print(f"No data found for segment {label} ({start_time}-{end_time}s)")

def plot_segments_html(data, segments):
    for start_time, end_time, label in segments:
        print(f"\nProcessing segment {label} ({start_time}-{end_time}s)")
        
        mask = (data['Time'] >= start_time) & (data['Time'] <= end_time)
        segment_data = data[mask].copy()
        
        if not segment_data.empty:  
            fig = go.Figure()
            
            trace = go.Scatter(
                x=segment_data['Time'].values,
                y=segment_data['Current'].values,
                mode='lines',
                name=label
            )
            
            fig.add_trace(trace)
            
            fig.update_layout(
                title=f'Test: {label}',
                xaxis_title='Time (s)',
                yaxis_title='Current (A)',
                showlegend=True,
                width=1200,
                height=400,
                margin=dict(l=50, r=50, t=50, b=50)
            )
            
            output_file = f"segment_{label.lower()}.html"
            fig.write_html(output_file)

def main():
    # Combine all data
    print("Combining all data...")
    combined_data = combine_all_data()
    
    print("\nCombined data summary:")
    print(f"Total points: {len(combined_data)}")
    print(f"Time range: {combined_data['Time'].min():.6f} to {combined_data['Time'].max():.6f}")
    print(f"Number of unique time points: {combined_data['Time'].nunique()}")
    
    output_file = "combined_waveform_data.csv"
    combined_data.to_csv(output_file, index=False)
    print(f"\nCombined data saved to {output_file}")
    
    # Create individual plots for each segment
    print("\nCreating segment plots...")
    plot_segments(combined_data, segments)
    plot_segments_html(combined_data, segments)

if __name__ == "__main__":
    main()


In [ ]:
import numpy as np
# Estimating for incomplete plots
# SF7, CR1, Tx10
max_I = 0.077 # max current when Tx for copper occlusion
min_I = 0.009 # min current for copper occlusion
# Full SF7 time durations estimation
durations = np.array([365.49999, 365.49999, 365.49999, 365.49999, 398.99998999999997,
98.99998999999985, 365.4999899999998, 398.9999899999996,
365.49999, 398.9999899999996, 365.49999,
398.9999900000001, 364.99999000000023, 112.4999999999998,
457.4999899999996, 29.99999999999936])

max_power = max_I * 3.3
min_power = min_I * 3.3
all_I = np.array([min_power, max_power, min_power, max_power, min_power, max_power, min_power, max_power, min_power, max_power, 
                  min_power, max_power, min_power, max_power, min_power, max_power])
energy = (durations/1000)*all_I
total_energy = energy.sum()
print(energy)
print(total_energy)

# ,current,duration_ms,Power,Energy
# 257,0.04145261154220553,398.999989999993,0.13679361808927826,0.054580652249684884
# 258,0.009509155411353975,365.4999900000462,0.031380212857468115,0.011469467485603915
# 259,0.04149676357252121,398.999989999993,0.13693931978931997,0.05463878722654451
# 260,0.009552000774134789,365.49998999998934,0.031521602554644805,0.011521145418506315
# 261,0.041504126922542066,398.999989999993,0.13696361884438882,0.054648482549273986
# 262,0.009516518708257437,365.49998999998934,0.03140451173724954,0.011478348725919256
# 263,0.041612670504797666,398.9999900000498,0.1373218126658323,0.0547914018804558
# 264,0.009498447700364296,365.49998999998934,0.03134487741120218,0.011456552380345288
# 265,0.04139971011530399,105.14999999998054,0.13661904338050315,0.014365492411457247
# 266,0.009560028642987248,365.49998999998934,0.03154809452185792,0.011530828232257788
# 267,0.04172673550271172,398.999989999993,0.13769822715894867,0.05494159125943728
# 268,0.009522545491803277,365.49998999998934,0.031424400122950814,0.011485617930694187
# 269,0.04175801137533028,398.999989999993,0.13780143753858992,0.054982772199882034
# 270,0.009517296286149161,364.499990000013,0.03140707774429223,0.011447879523724148
# 271,0.041645176007866276,112.50000000001137,0.1374290808259587,0.015460771592921915
# 272,0.009486789998786995,457.4999899999739,0.03130640699599708,0.014322680887603776
# 273,0.04154616967213115,29.999989999964782,0.1371023599180328,0.004113069426512557